# December 04, 2021

https://adventofcode.com/2021/day/4

In [1]:
import pandas as pd
import numpy as np

# Read Data

In [ ]:
balls = pd.read_csv("data/2021/04.txt", sep=",", nrows=1, header=None)
balls = list( balls.iloc[0] )

In [ ]:
nlines = !wc data/2021/04.txt -l
nlines = int( nlines[0].split(" ")[0] )
nboards = int( (nlines-1)/6 )

In [ ]:
def read_boards():
    boards = []
    for i in range(nboards):
        bd = pd.read_csv( "data/2021/04.txt", sep="\s+", nrows=5, header = None, skiprows = i*6+2, skip_blank_lines=False )
        boards.append(bd)
    return boards

boards = read_boards()

# Part 1

In [2]:
# user 1000+num to indicate a found number.
# victory means min in a row or col is 1000+
# assumes all numbers between 0 and 999

MARKER = 1000
def call_num_board(bd, num):
    nc = bd.shape[1]
    for c in range(nc):
        check = [r for r,x in enumerate(bd[c]) if x == num]
        if len(check) > 0:
            bd[c][check[0]] = MARKER + num
            return bd, True

    return bd, False

def check_win_board(bd):
    cond = max( bd.apply(min, axis=1).append(bd.apply(min, axis=0)) )
    return cond >= MARKER

def call_num(boards, num):
    win = False
    for bidx, bd in enumerate(boards):
        bd, hit = call_num_board(bd, num)
        if hit:
            win = check_win_board(bd)
            if win:
                break

    if win:
        print("board", bidx, "wins!")
        display(bd)
        return bidx
    
    return None

def play_game(boards, balls):
    for ball in balls:
        #print("next ball:", ball)
        win_idx = call_num(boards, ball)
        if win_idx is not None:
            break

    if win_idx is None:
        print("no winner yet")
        return None
    else:
        print("wtg", win_idx, "!")
        return boards[win_idx], ball
    
def score_board( bd, ball ):
    nums = bd.values.flatten().tolist()
    return ball * sum( [x for x in nums if x < MARKER] )

In [ ]:
boards = read_boards()
winner, last_ball = play_game(boards, balls)

In [ ]:
score_board( winner, last_ball )

# Part 2

In [ ]:
def call_num2(boards, num):
    winners = []

    for bidx, bd in enumerate(boards):
        bd, hit = call_num_board(bd, num)

        # check if the board wins
        if hit:
            if check_win_board(bd):
                winners.append(bidx)

    # return ids of all winners
    return winners

def find_loser(boards, balls):
    for i , ball in enumerate(balls):
        #print("next ball:", ball)
        winners = call_num2(boards, ball)

        winners.sort(reverse=True)
        for bidx in winners:
            del boards[bidx]

        if len(boards) == 0:
            return boards, i
        
        assert len(boards) > 0, "tie for worst?!"

    print("what happened?")
    return None, None

def score_board( bd, ball ):
    nums = bd.values.flatten().tolist()
    return ball * sum( [x for x in nums if x < MARKER] )

In [ ]:
boards = read_boards()
loser, ball_id = find_loser(boards, balls)

In [ ]:
loser[0]

In [ ]:
loser, last_ball = play_game( loser, balls[(ball_id+1):] )

In [ ]:
score_board(loser, last_ball)

In [ ]:
42+98+20+81+61+56

In [ ]:
358*97